In [1]:
"""
    Maps Standings and Fixtures for building the Training Data
"""

'\n    Maps Standings and Fixtures for building the Training Data\n'

In [3]:
import json
import os
import pandas as pd

In [4]:
# This is just a test
df = pd.DataFrame()
fixture = 'fixture.json'
team_home = dirName + '720/720.json'
team_away = dirName + '529/529.json'

with open(fixture) as f:
    my_fixture = json.load(f)
with open(team_home) as f:
    standings_home = json.load(f)
    del standings_home['stats_away']
    my_fixture.update({"team_home": standings_home})
with open(team_away) as f:
    standings_away = json.load(f)
    del standings_home['stats_home']
    my_fixture.update({"team_away": standings_away})
df = df.append(json_normalize(my_fixture))
df.head()

NameError: name 'dirName' is not defined

In [16]:
for col in df.columns: 
    print(col)

goals_home
goals_away
team_home.team.id
team_home.team.name
team_home.rank
team_home.points
team_home.goals_diff
team_home.form
team_home.description
team_away.team.id
team_away.team.name
team_away.rank
team_away.points
team_away.goals_diff
team_away.form
team_away.description
team_away.stats_home.played
team_away.stats_home.wins
team_away.stats_home.draws
team_away.stats_home.lose
team_away.stats_home.goals_for
team_away.stats_home.avg_goals_for
team_away.stats_home.goals_against
team_away.stats_home.avg_goals_against
team_away.stats_home.clean_sheet
team_away.stats_home.failed_to_score
team_away.stats_home.streaks.wins
team_away.stats_home.streaks.draws
team_away.stats_home.streaks.loses
team_away.stats_home.streaks.best_win
team_away.stats_home.streaks.best_lose
team_away.stats_home.streaks.best_goals_for
team_away.stats_home.streaks.best_goals_against
team_away.stats_away.played
team_away.stats_away.wins
team_away.stats_away.draws
team_away.stats_away.lose
team_away.stats_away.goal

In [4]:
league = '140'
season = '2019'

In [5]:
# League directory
dirName = league + '/' + season + '/' 

In [5]:
# Multi League directory
country = 'GB'
season = '2019'
dirCountry = '../' + country + '/'

In [9]:
# Historicity: multi season directory
country = 'US'
dirCountry = '../' + country + '/'
year_window = 2
start_season = 2020
seasons = list(range(start_season, start_season - year_window, -1))
print(seasons)

[2020, 2019]


In [10]:
df = pd.DataFrame()
standings = None
away_team_stands = None
fixture = None
# This will prevent duplicated fixtures
visited_fixtures = []
# This will help to map to standings
visited_standings = {}

# Principal leagues are sorted ascendent
dirLeague = sorted(os.listdir(dirCountry))

for league in dirLeague:
    if league == ".DS_Store":
        continue
    if league == "teams.json":
        continue
    for season in seasons:
        dirName = dirCountry + league + '/' + str(season) + '/' 
        for subdir in os.listdir(dirName):
            if subdir == ".DS_Store":
                continue
            team_id = subdir
            # Entering directory of a team
            dirTeam = dirName + team_id + '/'

            # Gets standings from a team if they have not being added
            if team_id not in visited_standings:
                json_standings = dirTeam + team_id + '.json'
                with open(json_standings) as f:
                    standings = json.load(f)
                    visited_standings[team_id] = standings
            else:
                standings = visited_standings[team_id]

            # Directory of team in home
            dirHome = dirTeam + 'home/'
            for file in os.listdir(dirHome):
                # Check if fixture hasn't being added to dataframe
                if file not in visited_fixtures:
                    path = dirHome + file
                    with open(path) as f:
                        fixture = json.load(f)
                        # Adds id of fixture as "id_fixture.json"
                        visited_fixtures.append(file)
                        # Next, get away team's external info
                        away_team = str(fixture["team_away"]["id"])
                        if away_team in visited_standings:
                            away_team_stands = visited_standings[away_team]
                        else:
                            # Get standings from away_team
                            fileAwayTeam = dirName + away_team + '/' + away_team + '.json'
                            with open(fileAwayTeam) as f:
                                away_team_stands = json.load(f)
                                visited_standings[away_team] = away_team_stands
                    # Three main jsons to a dataframe
                    # standings "this is the home team"
                    # away_team_stands
                    # the fixture itself
                    # The premise that away standings from home team aren't relevant
                    try:
                        #del standings['stats_away']
                        fixture.update({"team_home": standings})
                    except:
                        print("Couldn't delete stats away")
                    # The premise that home standings from away team aren't relevant
                    try:
                        #del away_team_stands['stats_home']
                        fixture.update({"team_away": away_team_stands})
                    except:
                        print("Couldn't delete stats home")
                    # Adding league
                    fixture.update({"league": int(league)})
                    # Adding season
                    fixture.update({"season": season})
                    df = df.append(json_normalize(fixture))
                else:
                    continue

            # Directory of team in away
            dirAway = dirTeam + 'away/'
            for file in os.listdir(dirHome):
                # Check if fixture hasn't being added to dataframe
                if file not in visited_fixtures:
                    path = dirAway + file
                    with open(dirAway) as f:
                        fixture = json.load(f)
                        # Adds id of fixture as "id_fixture.json"
                        visited_fixtures.append(file)
                        # Next, get home team's external info
                        home_team = str(fixture["team_home"]["id"])
                        if home_team in visited_standings:
                            home_team_stands = visited_standings[home_team]
                        else:
                            # Get standings from home_team
                            fileHomeTeam = dirName + home_team + '/' + home_team + '.json'
                            with open(fileHomeTeam) as f:
                                home_team_stands = json.load(f)
                                visited_standings[home_team] = home_team_stands
                    # Three main jsons to a dataframe
                    # standings "this is the away team"
                    # home_team_stands
                    # the fixture itself
                    # The premise that away standings from home team aren't relevant
                    try:
                        #del home_team_stands['stats_away']
                        fixture.update({"team_home": home_team_stands})
                    except:
                        print("Couldn't delete stats away")
                    # The premise that home standings from away team aren't relevant
                    try:
                        #del standings['stats_home']
                        fixture.update({"team_away": standings})
                    except:
                        print("Couldn't delete stats home")
                    # Adding league
                    fixture.update({"league": int(league)})
                    # Adding season
                    fixture.update({"season": season})
                    df = df.append(json_normalize(fixture))
                else:
                    continue

In [11]:
df.head()

,goals_home,goals_away,league,season,team_home.team.id,team_home.team.name,team_home.rank,team_home.points,team_home.goals_diff,team_home.form,...,stats_away.offside,stats_away.possession,stats_away.c_yellow,stats_away.c_red,stats_away.saves,stats_away.p_total,stats_away.p_accurate,stats_away.p_percentage,stats_home,stats_away
0,1,1,253,2020,1600,Houston Dynamo,12,1,-4,LD,...,2,41%,1,None,1,337,254,75%,NaN,NaN
0,None,None,253,2020,1607,Chicago Fire,7,1,-1,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,2,1,253,2020,1607,Chicago Fire,7,1,-1,DL,...,4,61%,3,None,3,593,537,91%,NaN,NaN
0,0,2,253,2020,1607,Chicago Fire,7,1,-1,DL,...,1,64%,3,None,3,475,382,80%,NaN,NaN
0,1,1,253,2020,1609,New England Revolution,8,1,-1,DL,...,None,56%,1,None,2,443,321,72%,NaN,NaN


In [12]:
df.shape

(483, 122)

In [13]:
# csv to save
file_title = 'mls.csv'
csv_file = dirCountry + file_title

In [14]:
df.to_csv(csv_file)

In [51]:
os.listdir(dirName)

['533',
 '798',
 '532',
 '.DS_Store',
 '543',
 '715',
 '529',
 '545',
 '542',
 '537',
 '530',
 '539',
 '538',
 '531',
 '536',
 '540',
 '541',
 '546',
 '548',
 '720',
 '727']